# 使用 milvas 测试小说《围城》数据集

## 准备

In [1]:
%%time

# 导入需要的包

from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.llms.openai_like import OpenAILike

from milvus import default_server
from pymilvus import connections, utility
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore

CPU times: user 3.54 s, sys: 492 ms, total: 4.03 s
Wall time: 3.67 s


In [2]:
Settings.chunk_size=128
Settings.chunk_overlap=10

Settings

_Settings(_llm=None, _embed_model=None, _callback_manager=None, _tokenizer=None, _node_parser=SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f77bb0c8f40>, id_func=<function default_id_func at 0x7f7880f13010>, chunk_size=128, chunk_overlap=10, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?'), _prompt_helper=None, _transformations=None)

In [3]:
%%time

# 初始化全局 llm

llm = OpenAILike(model="xiaoyu", 
                 api_base="http://192.168.0.72:3000/v1", 
                 api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
                 is_chat_model=True,
                 temperature=0.1
                )

Settings.llm =llm

CPU times: user 123 ms, sys: 0 ns, total: 123 ms
Wall time: 123 ms


In [4]:
%%time

# 初始化全局 embedding 模型

ollama_embedding = OllamaEmbedding(
    model_name="dztech/bge-large-zh:v1.5",
    # model_name="bge-m3:latest",
    base_url="http://192.168.0.72:11435",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.embed_model = ollama_embedding

CPU times: user 413 ms, sys: 8.04 ms, total: 421 ms
Wall time: 421 ms


In [5]:
%%time

import logging
import sys

# 设置系统日志，便于设置level排查

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

CPU times: user 387 µs, sys: 47 µs, total: 434 µs
Wall time: 438 µs


In [6]:
%%time

# 启动milvus服务器

default_server.set_base_dir('milvus_data')

# (OPTIONAL) if you want cleanup previous data
default_server.cleanup()

# Start your milvus server
default_server.start()

# Now you could connect with localhost and the given port
# Port is defined by default_server.listen_port
connections.connect(host='127.0.0.1', port=default_server.listen_port)

# Check if the server is ready.
print(utility.get_server_version())

# Stop your milvus server
# default_server.stop()

default_server.listen_port

v2.3.5-lite
CPU times: user 3.18 s, sys: 312 ms, total: 3.5 s
Wall time: 6.71 s


19530

## 索引

In [7]:
%%time

# load documents
documents = SimpleDirectoryReader("./books/").load_data()
print("Document ID:", documents[0].doc_id)

Document ID: 69f635fb-c52d-49e0-bac7-8da506da7e40
CPU times: user 10.6 ms, sys: 0 ns, total: 10.6 ms
Wall time: 10.2 ms


In [8]:
%%time

vector_store = MilvusVectorStore(dim=1024, overwrite=True)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: e603ee0fd0874fd0a9f911545f687abf
Created new connection using: e603ee0fd0874fd0a9f911545f687abf
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: llamacollection
Successfully created collection: llamacollection
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: llamacollection
Successfully created an index on collection: llamacollection
CPU times: user 16.7 s, sys: 707 ms, total: 17.4 s
Wall time: 1min 40s


## 查询

### 基本嵌入查询

In [9]:
%%time

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    similarity_cutoff=0.5
)

CPU times: user 37.2 ms, sys: 4.07 ms, total: 41.3 ms
Wall time: 40.7 ms


In [10]:
%%time

streaming_response = query_engine.query("方鸿渐的妻子是谁")
streaming_response.print_response_stream()
print()

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
孙柔嘉
CPU times: user 141 ms, sys: 4.63 ms, total: 146 ms
Wall time: 12.3 s


In [11]:
%%time

streaming_response = query_engine.query("方鸿渐的父亲是谁，说出他的名字")
streaming_response.print_response_stream()
print()

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
方豚翁
CPU times: user 94.1 ms, sys: 4.65 ms, total: 98.7 ms
Wall time: 9.47 s


In [12]:
%%time

streaming_response = query_engine.query("方鸿渐的父亲是谁，说出他的名字，名字好像带一个翁字")
streaming_response.print_response_stream()
print()

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
方鸿渐的父亲名字是方翁（辶＋豚）。
CPU times: user 122 ms, sys: 4.39 ms, total: 126 ms
Wall time: 11.5 s


In [13]:
%%time

streaming_response = query_engine.query("文中提到的局部真理是啥意思")
streaming_response.print_response_stream()
print()

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.16

### rerank

In [14]:
%%time

from llama_index.core.postprocessor import SentenceTransformerRerank

reranker = SentenceTransformerRerank(model='/models/bge-reranker-v2-m3', top_n=5)

CPU times: user 800 ms, sys: 708 ms, total: 1.51 s
Wall time: 803 ms


In [15]:
%%time

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100,
    node_postprocessors=[reranker],
    similarity_cutoff=0.5
)

CPU times: user 191 µs, sys: 0 ns, total: 191 µs
Wall time: 194 µs


In [16]:
%%time

streaming_response = query_engine.query("方鸿渐的妻子是谁")
streaming_response.print_response_stream()
print()

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
方鸿渐的妻子是孙柔嘉。
CPU times: user 1.45 s, sys: 92 ms, total: 1.54 s
Wall time: 2.9 s


In [17]:
%%time

streaming_response = query_engine.query("方鸿渐的父亲是谁，说出他的名字")
streaming_response.print_response_stream()
print()

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
方鸿渐的父亲的名字在提供的文本中没有直接提及。不过，从上下文可以推断，他是苏鸿业、方弼和方相的长辈，以及柔嘉口中提到的“方家”的家长。由于没有具体名字，我无法直接给出答案。
CPU times: user 859 ms, sys: 24.1 ms, total: 883 ms
Wall time: 4.08 s


In [18]:
%%time

streaming_response = query_engine.query("方鸿渐的父亲是谁，说出他的名字，名字好像带一个翁字")
streaming_response.print_response_stream()
print()

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
方鸿渐的父亲的名字是方遯翁。
CPU times: user 844 ms, sys: 7.38 ms, total: 851 ms
Wall time: 2.33 s


In [19]:
%%time

streaming_response = query_engine.query("文中提到的局部真理是啥意思")
streaming_response.print_response_stream()
print()

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST http://192.168.0.72:3000/v1/chat/completions "HTTP/1.1 200 OK"
文中提到的“局部的真理”可能是指在特定情境或语境下，某个陈述或观点虽然不全面，但却是部分正确的。鲍小姐的例子中，可能是因为她并未完全赤裸，所以他们将一个过于直接或绝对的说法修正为更符合实际情况的“局部的真理”。这表明在讨论中，人们会根据实际情况调整语言，以避免误解或过于极端。
CPU times: user 898 ms, sys: 27 ms, total: 925 ms
Wall time: 4.82 s
